## Library Loadings

In [1]:
import boto
from boto.manage.cmdshell import sshclient_from_instance
import time
import os

## S3 Bucket Connections

In [2]:
s3 = boto.connect_s3(aws_access_key_id='', 
                     aws_secret_access_key='')

## EC2 Connection and script launch

##### Connection to ec2

In [3]:
ec2 = boto.connect_ec2(aws_access_key_id='', 
                       aws_secret_access_key='')

In [4]:
# ##To do Only Once : 
#key_pair = ec2.create_key_pair('ec2-romain2-key')
#key_pair.save('~/.ssh')
#group = ec2.create_security_group('root', 'A group that allows SSH access')
#group.authorize('tcp', 22, 22, '0.0.0.0/0') # to properly authorize ssh

##### Run Instance

In [5]:
## Running a classical Amazon t2.micro Linux Instance
reservation = ec2.run_instances(image_id='ami-60b6c60a', 
                                key_name='ec2-romain2-key', 
                                instance_type = 't2.micro',
                                security_groups = ['root'])

instance = reservation.instances[0]
print 'waiting for instance'
while instance.state != 'running':
    print '.',
    time.sleep(5)
    instance.update()
print 'done'

waiting for instance
. . . . . . . done


##### Upload Script

In [6]:
import boto.manage.cmdshell
 
def upload_file(instance, key, username, local_filepath, remote_filepath):
    """
    Upload a file to a remote directory using SFTP. All parameters except
    for "instance" are strings. The instance parameter should be a
    boto.ec2.instance.Instance object.
 
    instance        An EC2 instance to upload the files to.
    key             The file path for a valid SSH key which can be used to
                    log in to the EC2 machine.
    username        The username to log in as.
    local_filepath  The path to the file to upload.
    remote_filepath The path where the file should be uploaded to.
    """
    ssh_client = boto.manage.cmdshell.sshclient_from_instance(
        instance,
        key,
        user_name=username
    )
    ssh_client.put_file(local_filepath, remote_filepath)

In [7]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain2-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical2/install.sh', 'install.sh')

In [8]:
upload_file(instance, 
            '/Users/romainbui/.ssh/ec2-romain2-key.pem', 
            'ec2-user', 
            '/Users/romainbui/Documents/Harvard/CS-181-Machine_Learning/Practicals_GIT/cs181-practicals/practical2/main.py', 'main.py')

## End EC2 : DO NOT FORGET

In [9]:
instance.terminate()